In [ ]:
# imports libraries and dependencies
from selenium import webdriver
import time
import pandas as pd
import random
from dotenv import load_dotenv
import os

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
# url used to get the internship postings which is already filtered
url1 = 'https://www.linkedin.com/jobs/search/?keywords=Software%20Engineer%20Intern&location=United%20States&geoId=103644278&f_TPR=r604800&position=1&pageNum=0'

In [ ]:
# sets up the chrome driver and opens the webpage
load_dotenv()
service = Service(os.getenv("CHROMEDRIVER_PATH"))
driver = webdriver.Chrome(service=service)
driver.get(url1)
driver.implicitly_wait(10)

In [ ]:
# close popup
actions = ActionChains(driver)
actions.move_by_offset(random.randint(1, 5), random.randint(1, 5)).click().perform()

In [ ]:
# gets the number of postings
n = driver.find_element(By.CLASS_NAME, "results-context-header__job-count").text

In [ ]:
n = pd.to_numeric(n)
n

In [ ]:
# scrolls through the page until the last posting is available
i = 1
numScroll = (n/25)+1

while(i <= numScroll):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    
    try:
        see_more_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='See more jobs']"))
        )
        see_more_button.click()
        time.sleep(random.randint(3, 6))
    except:
        pass
        time.sleep(random.randint(2, 5))

In [23]:
companyName = []

In [ ]:
# gets the company names of all postings
numJobs = 0

for j in range(n):
    try:
        company = driver.find_elements(By.CLASS_NAME, "base-search-card__subtitle")[j].text
        numJobs = numJobs+1
        companyName.append(company)
    
    except IndexError:
        pass

In [ ]:
len(companyName)

In [ ]:
titleName = []

In [ ]:
# gets the posting title of each internship
for i in range(numJobs):
    try:
        title = driver.find_elements(By.CLASS_NAME, "base-search-card__title")[i].text
        titleName.append(title)
    
    except IndexError:
        pass

In [ ]:
len(titleName)

In [29]:
linkList = []

In [ ]:
# gets the links of all internships
for k in range(n):
    try:
        link_element = driver.find_elements(By.CLASS_NAME, "base-card__full-link")[k].get_attribute('href')
        if link_element is not None:
            linkList.append(link_element)
    except IndexError:
        pass

In [ ]:
len(linkList)

In [ ]:
# gets indices of hidden or empty links
remainingLinks = []
indexList = []

for k in range(n):
    try:
        link_element = driver.find_elements(By.CLASS_NAME, "base-card--link")[k].get_attribute('href')
        if link_element is not None:
            indexList.append(k)
            remainingLinks.append(link_element)
    except IndexError:
        pass

In [ ]:
# uses the indices to fill the empty/hidden links
for i in indexList:
    linkList.insert(i, remainingLinks[0])
    remainingLinks.pop(0)

In [34]:
indexList

[]

In [35]:
datePosted = []

In [ ]:
# gets the date of all postings
for m in range(n):
    try:
        time_element = driver.find_elements(By.CSS_SELECTOR, ".job-search-card__listdate, .job-search-card__listdate--new")[m].get_attribute('datetime')
        datePosted.append(time_element)
    except IndexError:
        pass

In [ ]:
len(datePosted)

In [ ]:
# convert all data to pandas dataframe
companyData = pd.DataFrame(companyName, columns=["Company"])
titleData = pd.DataFrame(titleName, columns=["Title"])
linkData = pd.DataFrame(linkList, columns=["Links"])
dateData = pd.DataFrame(datePosted, columns=["Date Posted"])

In [ ]:
# processes all data in order to be converted to a CSV file
jobData = pd.DataFrame()
jobData = pd.concat([companyData, titleData, dateData, linkData], axis=1)
numColumns = (jobData.shape[1]*2)-1

for i in range(1, numColumns, 2):
    jobData.insert(i, f'Empty_{i}', '')
    
jobData.index = jobData.index+1

In [ ]:
jobData

In [ ]:
# puts all the postings in a csv file
jobData.to_csv("LinkedIn_SWE_Intern_Jobs.csv")

In [45]:
driver.quit()